In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.decomposition import FastICA
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.ensemble import IsolationForest
from sklearn.cluster import DBSCAN
import umap
import hdbscan

d:\Apps\anaconda3\envs\poop\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Wczytanie danych

In [2]:
hc_training = pd.read_csv('../../../classifier_data/HC_training.csv', sep='\t').T
hc_test = pd.read_csv('../../../classifier_data/HC_test.csv', sep='\t').T
oc_training = pd.read_csv('../../../classifier_data/OC_training.csv', sep='\t').T
oc_test = pd.read_csv('../../../classifier_data/OC_test.csv', sep='\t').T

synthetic_hc_data_path = '../../../best_models/HC/WGAN/50/synthetic_data/'
model_id = os.listdir(synthetic_hc_data_path)[0]
hc_synthetic = pd.read_csv(f'{synthetic_hc_data_path}/{model_id}/generated_data.tsv', sep='\t')

synthetic_oc_data_path = '../../../best_models/OC/WGAN/50/synthetic_data/'
model_id = os.listdir(synthetic_oc_data_path)[0]
oc_synthetic = pd.read_csv(f'{synthetic_oc_data_path}/{model_id}/generated_data.tsv', sep='\t')

# combine training and test data with labels
hc_training['label'] = 'HC'
hc_test['label'] = 'HC'
oc_training['label'] = 'OC'
oc_test['label'] = 'OC'

hc_synthetic['label'] = 'HC'
oc_synthetic['label'] = 'OC'

training_data = pd.concat([hc_training, oc_training])
test_data = pd.concat([hc_test, oc_test])

In [3]:
# keep only the columns that are in both datasets
# Znalezienie wspólnych kolumn
common_columns = training_data.columns.intersection(test_data.columns)
common_columns = common_columns.intersection(hc_synthetic.columns)
common_columns = common_columns.intersection(oc_synthetic.columns)

# Uporządkowanie kolumn w real_data
training_data = training_data[common_columns]

# Uporządkowanie kolumn w synthetic_data
test_data = test_data[common_columns]

# Uporządkowanie kolumn w synthetic_data
hc_synthetic = hc_synthetic[common_columns]
oc_synthetic = oc_synthetic[common_columns]

common_columns

Index(['ENSG00000019582', 'ENSG00000054654', 'ENSG00000076928',
       'ENSG00000081237', 'ENSG00000085265', 'ENSG00000090382',
       'ENSG00000106066', 'ENSG00000110719', 'ENSG00000115523',
       'ENSG00000119535', 'ENSG00000120129', 'ENSG00000121316',
       'ENSG00000135046', 'ENSG00000136167', 'ENSG00000137642',
       'ENSG00000147168', 'ENSG00000160255', 'ENSG00000163131',
       'ENSG00000177359', 'ENSG00000184319', 'ENSG00000196126',
       'ENSG00000198336', 'ENSG00000211899', 'ENSG00000240356',
       'ENSG00000244734', 'ENSG00000257207', 'ENSG00000269028', 'label'],
      dtype='object')

In [4]:
# add synthetic data to training data
training_data_synthetic = pd.concat([training_data, hc_synthetic, oc_synthetic])
training_data_synthetic

,ENSG00000019582,ENSG00000054654,ENSG00000076928,ENSG00000081237,ENSG00000085265,ENSG00000090382,ENSG00000106066,ENSG00000110719,ENSG00000115523,ENSG00000119535,...,ENSG00000177359,ENSG00000184319,ENSG00000196126,ENSG00000198336,ENSG00000211899,ENSG00000240356,ENSG00000244734,ENSG00000257207,ENSG00000269028,label
VUMC-HC-0033-TR2591,7.136091,5.318681,5.395822,6.121758,4.078496,4.643048,4.211676,4.169415,5.849773,4.399111,...,3.083092,3.535280,4.029213,7.670810,4.911210,3.535280,6.995484,9.847484,7.757440,HC
Vumc-HD-70-TR1062,9.178061,6.106566,6.461376,9.448584,4.926369,5.721557,4.653637,5.309622,7.717082,4.219669,...,5.079361,3.701701,5.786592,7.237932,6.633504,3.701701,10.021953,8.262227,6.645036,HC
VUMC-HC0053-DOT-HD-48h-TR3087,7.631874,5.383165,5.975680,7.317863,4.845811,4.997802,3.083092,5.151550,5.593316,3.083092,...,3.083092,6.754951,4.529534,3.720811,4.559495,7.783065,7.050183,8.831693,4.253528,HC
Vumc-HD-149-TR932,6.897237,3.647372,5.524362,5.552469,4.492627,4.275620,4.459443,3.544794,5.451417,3.083092,...,3.544794,8.271851,4.616154,3.083092,4.389800,9.503594,8.661739,3.647372,8.517492,HC
Vumc-HD-36-TR1165,8.796382,7.775699,7.918580,9.002749,6.525333,5.811822,4.590595,7.579274,7.306126,5.743631,...,4.172174,4.893418,6.813860,7.438673,6.525333,5.671874,9.877834,3.862107,7.025844,HC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,6.369244,5.166058,4.510162,5.695530,4.200341,3.404419,3.217804,5.155252,7.059708,3.515179,...,8.620451,6.339598,4.967874,3.164897,4.020642,7.706444,10.657877,7.761838,6.274260,OC
77,9.825571,8.882399,9.022001,10.389135,7.238414,5.831624,6.532637,8.425561,9.524918,6.111047,...,4.644109,8.015701,7.968525,7.639619,7.642188,9.484810,10.560298,5.135900,4.140117,OC
78,7.935232,6.915240,6.398023,7.297281,4.573359,4.305329,3.690265,6.932302,8.374109,5.405024,...,4.482417,4.702991,5.040316,3.519573,4.404007,5.522831,9.436274,9.543081,6.143652,OC
79,7.057655,4.951875,5.166376,5.683837,4.745713,6.420458,4.383182,4.757803,5.881359,4.264245,...,7.104145,6.255760,4.228486,3.305400,4.594848,6.690172,7.986446,8.524309,5.853624,OC


### Wykorzystanie klasyfikatorów

In [5]:
# klasyfikacja z uyciem svm
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score


def svm_classification(training_data, test_data):
    # Oddziel etykiety od cech
    X_train = training_data.drop('label', axis=1)
    y_train = training_data['label']
    X_test = test_data.drop('label', axis=1)
    y_test = test_data['label']

    # Normalizacja danych
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Klasyfikacja przy użyciu SVM
    svm = SVC(kernel='linear', random_state=42)
    svm.fit(X_train, y_train)
    y_pred = svm.predict(X_test)

    # Wypisanie dokładności i innych metryk
    print(f"Dokładność: {accuracy_score(y_test, y_pred)}")
    print(classification_report(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))


svm_classification(training_data, test_data)
svm_classification(training_data_synthetic, test_data)

Dokładność: 0.8162162162162162
              precision    recall  f1-score   support

          HC       0.83      0.94      0.88       131
          OC       0.78      0.52      0.62        54

    accuracy                           0.82       185
   macro avg       0.80      0.73      0.75       185
weighted avg       0.81      0.82      0.80       185

[[123   8]
 [ 26  28]]
Dokładność: 0.8108108108108109
              precision    recall  f1-score   support

          HC       0.84      0.91      0.87       131
          OC       0.72      0.57      0.64        54

    accuracy                           0.81       185
   macro avg       0.78      0.74      0.76       185
weighted avg       0.80      0.81      0.80       185

[[119  12]
 [ 23  31]]


In [6]:
from sklearn.tree import DecisionTreeClassifier
# klasyfikacja z uyciem random forest
def decision_tree_classification(training_data, test_data):
    # Oddziel etykiety od cech
    X_train = training_data.drop('label', axis=1)
    y_train = training_data['label']
    X_test = test_data.drop('label', axis=1)
    y_test = test_data['label']

    # Normalizacja danych
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    # Klasyfikacja
    dt = DecisionTreeClassifier(max_depth=5, random_state=42)
    dt.fit(X_train, y_train)
    y_pred = dt.predict(X_test)

    # Wypisanie dokładności i innych metryk
    print(f"Dokładność: {accuracy_score(y_test, y_pred)}")
    print(classification_report(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))

decision_tree_classification(training_data, test_data)
decision_tree_classification(training_data_synthetic, test_data)

Dokładność: 0.7567567567567568
              precision    recall  f1-score   support

          HC       0.79      0.90      0.84       131
          OC       0.63      0.41      0.49        54

    accuracy                           0.76       185
   macro avg       0.71      0.65      0.67       185
weighted avg       0.74      0.76      0.74       185

[[118  13]
 [ 32  22]]
Dokładność: 0.7351351351351352
              precision    recall  f1-score   support

          HC       0.77      0.89      0.83       131
          OC       0.58      0.35      0.44        54

    accuracy                           0.74       185
   macro avg       0.67      0.62      0.63       185
weighted avg       0.71      0.74      0.71       185

[[117  14]
 [ 35  19]]


In [7]:
from sklearn.ensemble import RandomForestClassifier
# klasyfikacja z uyciem random forest
def random_forest_classification(training_data, test_data):
    # Oddziel etykiety od cech
    X_train = training_data.drop('label', axis=1)
    y_train = training_data['label']
    X_test = test_data.drop('label', axis=1)
    y_test = test_data['label']

    # Normalizacja danych
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    # Klasyfikacja
    rf = RandomForestClassifier(n_estimators=100, random_state=42)
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_test)

    # Wypisanie dokładności i innych metryk
    print(f"Dokładność: {accuracy_score(y_test, y_pred)}")
    print(classification_report(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))

random_forest_classification(training_data, test_data)
random_forest_classification(training_data_synthetic, test_data)

Dokładność: 0.7675675675675676
              precision    recall  f1-score   support

          HC       0.77      0.95      0.85       131
          OC       0.74      0.31      0.44        54

    accuracy                           0.77       185
   macro avg       0.76      0.63      0.65       185
weighted avg       0.76      0.77      0.73       185

[[125   6]
 [ 37  17]]
Dokładność: 0.7891891891891892
              precision    recall  f1-score   support

          HC       0.80      0.93      0.86       131
          OC       0.73      0.44      0.55        54

    accuracy                           0.79       185
   macro avg       0.76      0.69      0.71       185
weighted avg       0.78      0.79      0.77       185

[[122   9]
 [ 30  24]]


In [8]:
from xgboost import XGBClassifier

# klasyfikacja z uyciem xgboost
def xgboost_classification(training_data, test_data):
    # Oddziel etykiety od cech
    X_train = training_data.drop('label', axis=1)
    y_train = training_data['label']
    X_test = test_data.drop('label', axis=1)
    y_test = test_data['label']

    # Normalizacja danych
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    # Klasyfikacja
    y_train = y_train.replace('HC', 0)
    y_train = y_train.replace('OC', 1)
    y_test = y_test.replace('HC', 0)
    y_test = y_test.replace('OC', 1)
    xgb = XGBClassifier(random_state=42)
    xgb.fit(X_train, y_train)
    y_pred = xgb.predict(X_test)

    # Wypisanie dokładności i innych metryk
    print(f"Dokładność: {accuracy_score(y_test, y_pred)}")
    print(classification_report(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))

xgboost_classification(training_data, test_data)
xgboost_classification(training_data_synthetic, test_data)

Dokładność: 0.8
              precision    recall  f1-score   support

           0       0.81      0.93      0.87       131
           1       0.74      0.48      0.58        54

    accuracy                           0.80       185
   macro avg       0.78      0.71      0.73       185
weighted avg       0.79      0.80      0.79       185

[[122   9]
 [ 28  26]]
Dokładność: 0.827027027027027
              precision    recall  f1-score   support

           0       0.84      0.93      0.88       131
           1       0.78      0.57      0.66        54

    accuracy                           0.83       185
   macro avg       0.81      0.75      0.77       185
weighted avg       0.82      0.83      0.82       185

[[122   9]
 [ 23  31]]


In [9]:
from sklearn.neural_network import MLPClassifier

# klasyfikacja z uzyciem sieci neuronowych

def nn_classification(training_data, test_data):
    # Oddziel etykiety od cech
    X_train = training_data.drop('label', axis=1)
    y_train = training_data['label']
    X_test = test_data.drop('label', axis=1)
    y_test = test_data['label']

    # Normalizacja danych
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    # Klasyfikacja
    nn = MLPClassifier(hidden_layer_sizes=(100,100), random_state=42)
    nn.fit(X_train, y_train)
    y_pred = nn.predict(X_test)

    # Wypisanie dokładności i innych metryk
    print(f"Dokładność: {accuracy_score(y_test, y_pred)}")
    print(classification_report(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))

nn_classification(training_data, test_data)
nn_classification(training_data_synthetic, test_data)

d:\Apps\anaconda3\envs\poop\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Dokładność: 0.8108108108108109
              precision    recall  f1-score   support

          HC       0.85      0.89      0.87       131
          OC       0.70      0.61      0.65        54

    accuracy                           0.81       185
   macro avg       0.77      0.75      0.76       185
weighted avg       0.81      0.81      0.81       185

[[117  14]
 [ 21  33]]
Dokładność: 0.8108108108108109
              precision    recall  f1-score   support

          HC       0.85      0.89      0.87       131
          OC       0.69      0.63      0.66        54

    accuracy                           0.81       185
   macro avg       0.77      0.76      0.76       185
weighted avg       0.81      0.81      0.81       185

[[116  15]
 [ 20  34]]
